__линейная регрессия с помощью TensorFlow__

Евгений Борисов borisov.e@solarl.ru

In [1]:
# https://medium.com/analytics-vidhya/tf-gradienttape-explained-for-keras-users-cc3f06276f22

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
print(tf.__version__)

2.3.1


In [4]:
# сгенерируем модельный пример

# параметры зависимости, которые требуется найти
a_train, b_train = 10.,5.

# зависимость, которую будем восстанавливать
f = lambda x: x*a_train + b_train

# данные, по которым будем восстанавливать зависимость
x_train = np.array(range(10))
y_train = f(x_train)

In [5]:
# реализуем метод градиентного спуска

# функция потери
def loss(real_y, pred_y): return tf.abs(real_y - pred_y)

# параметры, которые будем искать по данным
a_real = tf.Variable(np.random.rand(), trainable=True)
b_real = tf.Variable(np.random.rand(), trainable=True)

t_speed = .01 # коэффициент скорости обучения

# шаг процесса оптимизации 
def step( x_real, y_real ):
    with tf.GradientTape(persistent=True) as tape:
        # текущий результат
        pred_y = a_real * x_real + b_real
        # считаем потерю
        reg_loss = loss(y_real, pred_y)
    
    # значение градиента
    a_gradients, b_gradients = tape.gradient(reg_loss, (a_real, b_real))

    # изменяем параметры
    a_real.assign_sub(a_gradients * t_speed)
    b_real.assign_sub(b_gradients * t_speed)

# основной цикл обучения
for _ in range(1000): step(x_train, y_train)

In [6]:
# что получили
print(f'y ≈ {a_real.numpy()} * x + {b_real.numpy()}')

y ≈ 10.398397445678711 * x + 5.000351428985596


In [7]:
# что должно быть
print(f'y ≈ {a_train} * x + {b_train}')

y ≈ 10.0 * x + 5.0
